<a href="https://colab.research.google.com/github/RAVIKUMARBALIJA/PySparkTutorials/blob/main/pyspark_mllib_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 31.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b51a2b914b21e14f0fe8607cdb564c4fce8b603ac410f1b34cd8e3d082437a91
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('MLIntro').getOrCreate()

In [16]:
df=spark.read.csv('/content/sample_data/test4.csv',header=True,inferSchema=True,sep=',')

In [17]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [18]:
df.show()

+---------+---+----------+------+
|     Name|age|experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
feature_assembler=VectorAssembler(inputCols=['age','experience'],outputCol='independent features')

In [21]:
output=feature_assembler.transform(df)

In [22]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|experience|Salary|independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [24]:
type(output)

pyspark.sql.dataframe.DataFrame

In [25]:
output['independent features','Salary'].show()

+--------------------+------+
|independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [26]:
dataset=output['independent features','Salary']

In [27]:
from pyspark.ml.regression import LinearRegression

In [29]:
train_set,test_set=dataset.randomSplit([0.75,0.25])

In [30]:
ps_model=LinearRegression(featuresCol='independent features',labelCol='Salary')
ps_model=ps_model.fit(dataset)

In [31]:
ps_model

LinearRegressionModel: uid=LinearRegression_ea3cf80f167f, numFeatures=2

In [32]:
ps_model.coefficients

DenseVector([-115.8558, 1602.7301])

In [ ]:
ps_model.intercept

In [37]:
pred_results=ps_model.evaluate(test_set)

In [40]:
pred_results.predictions.show()

+--------------------+------+------------------+
|independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [30.0,8.0]| 25000|26250.962548127405|
|         [31.0,10.0]| 30000| 29340.56702835141|
+--------------------+------+------------------+



In [41]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(955.1977598879985, 999879.5704573515)

In [43]:
pred_results.r2

0.8400192687268238

In [44]:
pred_results.rootMeanSquaredError

999.9397834156572

In [46]:
import numpy as np

In [50]:
np.sqrt(99999999)

9999.99995